In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('customers_logistic_regression').getOrCreate()

24/01/12 19:20:50 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.25.126.180 instead (on interface eth0)
24/01/12 19:20:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 19:20:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/12 19:20:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
df = spark.read.csv('./data/ML/logistic/customer_churn.csv', inferSchema=True, header=True)

In [8]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)


In [11]:
df.show(50)

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [19]:
df.count()

900

In [20]:
df.select(['Location']).distinct().count()

900

In [21]:
df.select(['Company']).distinct().count()

873

In [22]:
df.select(['Onboard_date']).distinct().count()


900

In [23]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [24]:
chosen_cols = df.select([
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn'])

In [41]:
final_data = chosen_cols.dropna()

In [42]:
final_data.show()

+----+--------------+-----+---------+-----+
| Age|Total_Purchase|Years|Num_Sites|Churn|
+----+--------------+-----+---------+-----+
|42.0|       11066.8| 7.22|      8.0|    1|
|41.0|      11916.22|  6.5|     11.0|    1|
|38.0|      12884.75| 6.67|     12.0|    1|
|42.0|       8010.76| 6.71|     10.0|    1|
|37.0|       9191.58| 5.56|      9.0|    1|
|48.0|      10356.02| 5.12|      8.0|    1|
|44.0|      11331.58| 5.23|     11.0|    1|
|32.0|       9885.12| 6.92|      9.0|    1|
|43.0|       14062.6| 5.46|     11.0|    1|
|40.0|       8066.94| 7.11|     11.0|    1|
|30.0|      11575.37| 5.22|      8.0|    1|
|45.0|       8771.02| 6.64|     11.0|    1|
|45.0|       8988.67| 4.84|     11.0|    1|
|40.0|       8283.32|  5.1|     13.0|    1|
|41.0|       6569.87|  4.3|     11.0|    1|
|38.0|      10494.82| 6.81|     12.0|    1|
|45.0|       8213.41| 7.35|     11.0|    1|
|43.0|      11226.88| 8.08|     12.0|    1|
|53.0|       5515.09| 6.85|      8.0|    1|
|46.0|        8046.4| 5.69|     

In [29]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer)

In [60]:
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites'], outputCol='features')
output = assembler.transform(final_data)
output.show()

+----+--------------+-----+---------+-----+--------------------+
| Age|Total_Purchase|Years|Num_Sites|Churn|            features|
+----+--------------+-----+---------+-----+--------------------+
|42.0|       11066.8| 7.22|      8.0|    1|[42.0,11066.8,7.2...|
|41.0|      11916.22|  6.5|     11.0|    1|[41.0,11916.22,6....|
|38.0|      12884.75| 6.67|     12.0|    1|[38.0,12884.75,6....|
|42.0|       8010.76| 6.71|     10.0|    1|[42.0,8010.76,6.7...|
|37.0|       9191.58| 5.56|      9.0|    1|[37.0,9191.58,5.5...|
|48.0|      10356.02| 5.12|      8.0|    1|[48.0,10356.02,5....|
|44.0|      11331.58| 5.23|     11.0|    1|[44.0,11331.58,5....|
|32.0|       9885.12| 6.92|      9.0|    1|[32.0,9885.12,6.9...|
|43.0|       14062.6| 5.46|     11.0|    1|[43.0,14062.6,5.4...|
|40.0|       8066.94| 7.11|     11.0|    1|[40.0,8066.94,7.1...|
|30.0|      11575.37| 5.22|      8.0|    1|[30.0,11575.37,5....|
|45.0|       8771.02| 6.64|     11.0|    1|[45.0,8771.02,6.6...|
|45.0|       8988.67| 4.8

In [33]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [34]:
log_reg_customers = LogisticRegression(featuresCol='features', labelCol='Churn')

In [35]:
pipeline = Pipeline(stages=[assembler, log_reg_customers])

In [87]:
train_data, test_data = final_data.randomSplit([0.8, 0.2])

In [88]:
fit_model = pipeline.fit(train_data)

In [89]:
test_results = fit_model.transform(test_data)

In [90]:
test_results.select(['Churn', 'prediction']).show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+


In [91]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [92]:
my_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [93]:
AUC = my_evaluator.evaluate(test_results)
AUC

0.7365319865319866

In [94]:
new_customers = spark.read.csv('./data/ML/logistic/new_customers.csv', header=True, inferSchema=True)

In [95]:
results = fit_model.transform(new_customers)

In [96]:
results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,7.7...|[2.89442219926861...|[0.94757001654367...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,9.2...|[-6.2140507028118...|[0.00199711863770...|       